# Irene City
From ancient Greek, *Εἰρήνη*: Peace.

This city finds the signal pulses within the waveforms produced by the
detector or by diomira in the case of Monte Carlo data.

This includes a number of tasks:
- Remove the signal-derivative effect of the PMT waveforms.
- Calibrate PMTs and produced a PMT-summed waveform.
- Remove the baseline from the SiPM waveforms and calibrate them.
- Apply a threshold to the PMT-summed waveform.
- Find pulses in the PMT-summed waveform.
- Match the time window of the PMT pulse with those in the SiPMs.
- Build the PMap object.

In [42]:
import os
from os.path import join as join_path

### Constants

In [43]:
ICDIR     = os.environ["ICDIR"]
ICDATADIR = join_path(ICDIR, "database/test_data/")
ICOUTDIR  = "output/"

## Initialization

In [44]:
if not os.path.isdir(ICOUTDIR):
    os.mkdir(ICOUTDIR)

## City Summary

A city is an algorithm which takes a dictionary with a given number of inputs
and returns another dictionary with processed outputs, after doing a series
of tasks.

Here we have several examples for Irene City, taken from
`invisible_cities/cities/irene_test.py` test:

## Examples of Irene usage


### Electrons 40 keV

In file `electrons_40keV_ACTIVE_10evts_RWF.h5` there is data generated for
electrons with an energy of 40 keV.

Input and output path files are passed to the city as keys of the input `conf`
dictionary. The `ICDATADIR` is the directory with test data, and the `ICOUTDIR`
is our output directory:

In [45]:
print(f"ICDATADIR: {ICDATADIR}")
print(f"ICOUTDIR:  {ICOUTDIR}")

ICDATADIR: /home/mcruces/Documents/GitHub/IC/invisible_cities/database/test_data/
ICOUTDIR:  output/


So `files_in` and `file_out` keys of the `conf` dictionary are the variables:

In [46]:
input_path  = join_path(ICDATADIR, "electrons_40keV_ACTIVE_10evts_RWF.h5")
output_path = join_path(ICOUTDIR,  "electrons_40keV_ACTIVE_10evts_CWF.h5")


The `detector_db` key is the local database file, which is taken from the `DetDB` class in

In [47]:
from invisible_cities.database.load_db import DetDB

new_db = DetDB.new

The `event_range` key, in this case, from zero to the number of required events

In [48]:
nrequired = 2
evt_range = (0, nrequired)

This variables:

In [49]:
case = 1
thr_sipm_type  = ["common", "individual"][case]
thr_sipm_value = [ 3.5    ,  0.99       ][case]

And the last parameters passed through a list of objects to unpack
by `unpack_s12params()` function.

In [50]:
from invisible_cities.types.ic_types import minmax
import invisible_cities.core.system_of_units as units
# from invisible_cities.evm.ic_containers import S12Params as S12P
from missed import S12Params as S12P

s12params = [
    S12P(time   = minmax(min =  99 * units.mus,
                         max = 101 * units.mus),
         length = minmax(min =   4,
                         max =  20,),
         stride              =   4,
         rebin_stride        =   1),
    S12P(time   = minmax(min =    101 * units.mus,
                         max =   1199 * units.mus),
         length = minmax(min =     80,
                         max = 200000),
         stride              =     40,
         rebin_stride        =     40)
]


Once we have all parameters, we write the configuration with the `irene.conf` file

In [51]:
from invisible_cities.core.configure import configure
conf = configure(f"dummy {join_path(ICDIR, 'config/irene.conf')}".split())

and then update the configuration dictionary with our parameters previously calculated:

In [52]:
from missed import unpack_s12params

conf.update(dict(
    detector_db   = new_db,
    run_number    = 0,
    files_in      = input_path,
    file_out      = output_path,
    event_range   = evt_range,
    thr_sipm_type = thr_sipm_type,
    thr_sipm      = thr_sipm_value,
    **unpack_s12params(s12params)
))



Pass all parameters to Irene City

In [53]:
from invisible_cities.cities.irene import irene

cnt = irene(**conf)

events processed: 0, event number: 0
events processed: 1, event number: 1


which returns an `argparse.Namespace` instance with those entries:

In [54]:
d = vars(cnt)
for entry in d:
    print(f"- {entry}:\t{d[entry]}")


- events_in:	2
- events_out:	2
- evtnum_list:	[0, 1]
- over_thr:	PassedFailed(n_passed=2, n_failed=0)
- full_pmap:	PassedFailed(n_passed=2, n_failed=0)


### For a specific Run

Following similar steps:

In [58]:
run_number = 2983
nrequierd = 2
input_path  = join_path(ICDATADIR, "run_2983.h5")
output_path = join_path(ICOUTDIR,  "run_2983.h5")

conf = configure(f"dummy {join_path(ICDIR, 'config/irene.conf')}".split())
conf.update(dict(
    run_number    = run_number,
    files_in      = input_path,
    file_out      = output_path,
    event_range   = (0, nrequired),
    **unpack_s12params(s12params)
))

cnt = irene(**conf)

events processed: 0, event number: 1820002
events processed: 1, event number: 1820004


We take the output

In [59]:
d = vars(cnt)
for entry in d:
    print(f"- {entry}:\t{d[entry]}")



- events_in:	2
- events_out:	1
- evtnum_list:	[1820002]
- over_thr:	PassedFailed(n_passed=2, n_failed=0)
- full_pmap:	PassedFailed(n_passed=1, n_failed=1)


### Reading several files

Up to now, we have read files one by one

Test in line 226

